In [3]:
import pandas as pd

import random
import re
import pickle
import nltk

In [95]:
dataset = pd.read_csv("reviews.csv",sep="\t")
dataset.head(3)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,211.200.63.206
1,1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,192.17.224.99
2,2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,198.9.2.161


In [4]:
### WORKING ON TWITTER DATA FOR SENTIMENTAL ANALYSIS

with open("Pickle Files/classifier.pickle","rb") as f:
    clf = pickle.load(f)

with open("Pickle Files/TfidfModel.pickle","rb") as f:
    tfidf = pickle.load(f)

def getSentiment(text):

    # PREPROCESSING THE DATASET
    text = str(text)
    text = text.lower()
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"there's","there is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"who's","who is",text)
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"they're","they are",text)
    text = re.sub(r"who're","who are",text)
    text = re.sub(r"ain't","am not",text)
    text = re.sub(r"wouldn't","would not",text)
    text = re.sub(r"shouldn't","should not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"couldn't","could not",text)
    text = re.sub(r"won't","will not",text)
    
    text = re.sub(r"\W"," ",text)
    text = re.sub(r"\d"," ",text)
    text = re.sub(r"\s+[a-z]\s+"," ",text)
    text = re.sub(r"^[a-z]\s+"," ",text)    
    text = re.sub(r"\s+[a-z]$"," ",text)    
    text = re.sub(r"\s+"," ",text)    
    
    sent = clf.predict(tfidf.transform([text]).toarray())
    
    return sent[0]


## 1. different sentiment in review headline and review body

In [17]:
#1. different sentiment in review headline and review body

blocked_reviews = []
# stores the list of review_id of fake reviews

for i in range(len(dataset)):
    #iterate through the whole dataset
    
        if( getSentiment( dataset["review_headline"][i] ) != getSentiment( dataset["review_body"][i] ) ):
            # checking if the sentiment of the body and the headline are not same
            
            blocked_reviews.append(dataset["review_id"][i])
            # append review_id to the list of fake reviews.

In [18]:
blocked_reviews

['R31LOQ8JGLPRLK', 'RY8DDL22YG4R5', 'R3RRXU2R23NMQ9']

## 2. Users which are posting either all positive or negative reviews on different products of same brand


In [129]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [114]:
#2. Users which are posting either all positive or negative reviews on different products of same brand

customers = dataset.groupby("customer_id")
# groups dataset by customers

customer_list = dataset["customer_id"].unique()
#list of unique customers

blocked_users = []
# it will store list users that are writing fake reviews

size = len(customer_list.tolist())
#size of total unique customers

for i in range(size):
    # iterate through all the customers
    
    brand_df = customers.get_group(customer_list[i])    
    # Dataframe for each customers
    
    brands = brand_df.groupby("product_parent")
    # groups reviews of each customers by brand
    
    brands_list = brand_df["product_parent"].unique()
    # unique list of brands for each customers reviews
    
    no_of_brands = len(brands_list.tolist())
    # no. of brands for which reviews had been written by the customer
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of products for a brand for which a customer had written reviews
        
        no_of_products = len(product_df["product_id"])
        # no of products
        
        if no_of_products<=2:
            # it will filter the products which are less than 2 for a brand
            continue
            
        indices = product_df.index.values.tolist()
        # index of the dataframe of the products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][indices[0]])
        # sentiment of the review of the first product
        
        isSameSentiment = True
        
        for k in range(1,no_of_products):
            # iterate through all the roducts
            
            text = str(product_df["review_body"][indices[k]])
            # review of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment is different than discard it
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # if sentiments of all the products of same brand by a customer is same, 
            #append customer_id to blocked users list
            
            blocked_users.append(customer_list[i])
            break
        
    

In [115]:
blocked_users

[44855305,
 15532751,
 49648764,
 32038204,
 2830811,
 50027179,
 49796412,
 21714269,
 20737314,
 13051014,
 827710,
 47916002,
 49857940,
 32273935,
 13171749,
 948409,
 2371598,
 2346648,
 10462174,
 36917006,
 23943923,
 52870270,
 17957446,
 53017806,
 18163651,
 50014143,
 43856165,
 11577054,
 51949299,
 33107350,
 52803739,
 51666042,
 44834233,
 22429746,
 53090839,
 24110864,
 51346302,
 19276501,
 38985479,
 33034891,
 51321735,
 19619134,
 45070473,
 14225180,
 51712970,
 52228204,
 47618784,
 46711885,
 32900509,
 28171447,
 7394955,
 38781745,
 28171883,
 46502470,
 31481426,
 43748492,
 19880661,
 38933460,
 52690060,
 21932728,
 11215441,
 12323042,
 16752241,
 4478689,
 3378151,
 53037408,
 45733542,
 52460215,
 14215194,
 16975415,
 52938899,
 2332962,
 14999827,
 49338357,
 49509927,
 24831002,
 6670518,
 30766980,
 35314853,
 32770247,
 49266466,
 52455519,
 43131107,
 16378095,
 14339056,
 43964085,
 1560638,
 47996476,
 30468231,
 10203282,
 28961183,
 11995502,
 

In [121]:
'''
dataset.set_index("customer_id",inplace=True)
for customer in blocked_users:
    dataset.drop(customer,inplace=True)
dataset.reset_index()
'''

'\ndataset.set_index("customer_id",inplace=True)\nfor customer in blocked_users:\n    dataset.drop(customer,inplace=True)\ndataset.reset_index()\n'

## 3.Reviews from same IP either all positive or negative reviews on different products of same brand

In [122]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [130]:
#3.Reviews from same IP either all positive or negative reviews on different products of same brand


ip = dataset.groupby("IP Address")
#grouping the dataset by ip address

ip_list = dataset["IP Address"].unique()
#stores the list pf unique ip addresses

blocked_ip = []
#stores the list of ip address from where reviews have been written.

size = len(ip_list.tolist())
#stores the size of the total unique ip addresses

for i in range(size):
    # iterate through all the ip addresses
    
    brand_df = ip.get_group(ip_list[i])
    # Dataframe of brands for which reviews have been written from the same ip address
    
    brands = brand_df.groupby("product_parent")
    # grouping the products of the same brands for each ip addresses
    
    brands_list = brand_df["product_parent"].unique()
    #list of unique brands for each ip addresses
    
    no_of_brands = len(brands_list.tolist())
    # total no. of brands
    
    for j in range(no_of_brands):
        # iterate through all the brands
        
        product_df = brands.get_group(brands_list[j])
        # Dataframe of the products of each brand of each products
        
        no_of_products = len(product_df["product_id"])
        # no of products of each brand for each ip addresses
        
        if no_of_products<=2:
            # filter the reviews of the brandswith less than 3 reviews
            break
        
        indices = product_df.index.tolist()
        # indices of dataframe of products of each brand for each customers
        
        sentiment = getSentiment(product_df["review_body"][ indices[0] ])
        # sentiment of review of first product of each brand
                
        isSameSentiment = True
        
        for k in range(1,no_of_products):
            # iterate through all the reviews
            
            text = str(product_df["review_body"][indices[k]])
            # reviews of each product
            
            if getSentiment(text)!=sentiment :
                # if sentiment of 2 products of same brand are not same 
                # then check the next brand
                isSameSentiment = False
                break;
                
        if(isSameSentiment):
            # if all the sentiments are same , append ip to blocked list
            blocked_ip.append(ip_list[i])
            

In [131]:
blocked_ip

['193.93.167.87', '205.10.168.66']

In [53]:
'''
dataset.set_index("IP Address",inplace=True)
for ip in blocked_users:
    dataset.drop(ip,inplace=True)
dataset.reset_index()
'''

'\ndataset.set_index("IP Address",inplace=True)\nfor ip in blocked_users:\n    dataset.drop(ip,inplace=True)\ndataset.reset_index()\n'

## 4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

In [5]:
dataset.sort_values("customer_id",inplace=True)

In [8]:
dataset.head(1)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
24496,US,10071,R1XALU24Y932WC,B0048DLA90,864204912,NEW Click Wheel Flex Cable For Apple iPod Nano...,Mobile_Electronics,5,0.0,0.0,N,Y,Five Stars,Just what I needed. Thanks.,2014-07-31,217.163.92.95


In [139]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [144]:
#4. User posting (>3) reviews on the same day with all the reviews are either positive or negative.

customer_group = dataset.groupby("customer_id")
#creates the group of the customers 

customer_group_list = dataset["customer_id"].unique().tolist()
# list of unique customers

for i in range(len(customer_group_list)):
    # iterate through all customers , starts with 1 as column could not be included
    
    customer_reviews = customer_group.get_group( customer_group_list[i] )
    # Dataframe of data of each cutomers
    
    dates_list = customer_reviews["review_date"].unique().tolist()
    # list of dates of reviews written by each customers
    
    reviews_by_date = customer_reviews.groupby("review_date");
    # gouping reviews by date for each cutomers
    
    for j in range(len(dates_list)):
        # iterating through all dates
        
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        df = reviews_by_date.get_group(dates_list[j])
        #dataframe storing the details for each details for each customers
        
        indices = df.index.tolist()
        # indices of dataframe of each date
        
        for k in range(len(df)):
            # iterating through dataframe of each day for each customers
            
            text = df["review_body"][ indices[k] ]
            #review on a single day
            
            if(getSentiment(text) == 0):
                
                #if sentiment is negative, append review_id to list of negative reviews
                reviews_by_date_for_neg.append(df["review_id"][ indices[k] ])
                
            else:
                
                #if sentiment is positive, append review_id to list of positive reviews
                reviews_by_date_for_pos.append(df["review_id"][ indices[k] ])
                
        # CONDITION FOR CONSIDERING THE FAKE REVIEW
        
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        

6 ['R8MEA6IGAHO0B', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P', 'RRB9C05HDOD4O', 'R8MEA6IGAHO0B']
0 []
5 ['R31LOQ8JGLPRLK', 'R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P', 'R31LOQ8JGLPRLK']
1 ['RRB9C05HDOD4O']
2 ['R2Y0MM9YE6OP3P', 'R2Y0MM9YE6OP3P']
0 []
1 ['RRB9C05HDOD4O']
1 ['RRB9C05HDOD4O']
2 ['R26I2RI1GFV8QG', 'R26I2RI1GFV8QG']
0 []
2 ['RY8DDL22YG4R5', 'RY8DDL22YG4R5']
0 []
2 ['R2AT2426ZHFUHH', 'R2AT2426ZHFUHH']
0 []
1 ['R3RRXU2R23NMQ9']
0 []
1 ['R250PR8VJUZ62F']
0 []
1 ['RBEMQ29WJBHYG']
0 []
1 ['R19VVIUT4BZCMT']
0 []
0 []
1 ['R1DT8JJUQHUKUL']
1 ['RLIL6S9OGM3YA']
0 []
1 ['R34EZZ68VYPHO0']
0 []
0 []
1 ['R3TZNSOXS13SIG']
1 ['R12UBZT87UX3AP']
0 []
0 []
1 ['R5KCRE927NYMU']
1 ['RT12RQFU0V2AX']
0 []
0 []
1 ['R2K0YQPV5W2SJ1']
1 ['R104MY31OKNGXH']
0 []
1 ['R3EQYRGIIJQBHT']
0 []
1 ['R32MW6D6CXKCFW']
0 []
1 ['RFO6FJSAA391Z']
0 []
1 ['R1IG9KI4EWNE4W']
0 []
1 ['R3IF1H9L7WSGQD']
0 []
1 ['R38T4K7X8UBVO7']
0 []
1 ['R19C76D4LWB6CM']
0 []
0 []
1 ['R260G1C72KG6KC']
1 ['R2SLX39DAJSG0V']


1 ['R3AXWUF0IRZGWA']
0 []
0 []
1 ['R2MEKKYG0FQ9Y4']
0 []
1 ['RXWHLKFN27QXA']
1 ['R1MQJWZVLX625B']
0 []
1 ['RM4ZH3280XY7O']
0 []
0 []
1 ['R3PGIUX1EKQ4SZ']
1 ['R1PWLHUYDMJFYV']
0 []
0 []
1 ['R1OCN1MBFK15GL']
1 ['R30S1VPVHKH2MA']
0 []
1 ['R15KDYNJ3WR8RP']
0 []
1 ['R2IBYANGYFQ1V5']
0 []
1 ['RVOHUPZIPJAFP']
0 []
1 ['R33U59YX6A4OF7']
0 []
1 ['R39564H8KZKVJ0']
0 []
0 []
1 ['R38CWTC2QKHNS4']
1 ['RWDIQCDXM66ZA']
0 []
1 ['R16IHK139ZARUA']
0 []
1 ['RHOR6RVMS370J']
0 []
0 []
1 ['R3MGZ75QFPTB0S']
0 []
1 ['R30DV9DJ4YDSEH']
0 []
1 ['R1VMJENF098LBW']
1 ['R7L22LEG2BUVA']
0 []
0 []
1 ['R3QS7HA9QRDIZ9']
1 ['RRLR0UYQ5HEBQ']
0 []
0 []
1 ['R1JP6MNSIT0U2D']
1 ['R1UUWXSTSYBDRA']
0 []
1 ['R3TOTF7UMXG7PK']
0 []
0 []
1 ['R3MAHRZX4TVR7C']
1 ['R1AEL8XMKRT511']
0 []
0 []
1 ['R7WBC0JAA9UI']
1 ['R3BTDRVS0QUH91']
0 []
1 ['R2KXEAPUMU14IE']
0 []
1 ['R34Z5GSD6QVJC0']
0 []
0 []
1 ['R396KCIONQ5GMH']
0 []
1 ['R1RZE8IT05J6D8']
0 []
1 ['R3VJXA3GLDX8E9']
0 []
1 ['R2IJUP8JD5W160']
1 ['RYV0E9CUARKQO']
0 []
0 []
1 ['R30XZYR7CNQLJ

1 ['R7TRWT3D0NWVA']
0 []
1 ['R34C2S3GJBJDGG']
0 []
1 ['R115ACF5NH8VCT']
0 []
1 ['R2CNZSHYDNZLF']
0 []
1 ['R2B1J0DV4Q90C']
0 []
1 ['RTV7V99DW7LAR']
0 []
1 ['R19MCMNXXF3E5V']
0 []
1 ['R3QUX6UDEI7VEJ']
0 []
0 []
1 ['R2OHETRELQHI3L']
1 ['R1MU62FFMJ4URR']
0 []
1 ['R20EQ2PZ10UC5Q']
0 []
1 ['R1MUN0TZJC9MSA']
0 []
1 ['R2JR1SFS2V4RXR']
0 []
1 ['R2R51UWKLR517C']
0 []
1 ['R1NKGBEN0I4D01']
0 []
1 ['R1SDQJBNNFGHFW']
0 []
0 []
1 ['R1XJTQUONRQDDL']
1 ['R2V9RVGPI8U2FA']
0 []
1 ['R23WA4EC8WVMD9']
0 []
1 ['R3H4GH4VGRSNGJ']
0 []
1 ['R2H5CL0LU2KH3Q']
0 []
1 ['R26OXBIFZKCCI8']
0 []
1 ['R5M17UM979QXF']
0 []
1 ['RNN38RFJVIE5G']
0 []
1 ['R1DTSELYJEDK2M']
0 []
0 []
1 ['R1XOP8ERFM4B5E']
1 ['R3MWLDA9M77DJ5']
0 []
1 ['R1RPRSVD6QYAX1']
0 []
1 ['R1S6OGES5HTB55']
0 []
1 ['RTIOX61PKJVN1']
0 []
0 []
1 ['R877I6M0MKQ97']
1 ['R30WJHSATH8ESG']
0 []
1 ['RE0MBYEZJR5TA']
0 []
1 ['R1ZQMYGHOEUHL7']
0 []
1 ['R38O0DB0BLUWJT']
0 []
1 ['R2BB2MQN08YUHK']
0 []
1 ['R3V7GO2GYDJA9M']
0 []
0 []
1 ['R2E16WLZKP78GO']
0 []
1 ['R273W61K6FDZ

1 ['R3KX5WKLAVXPTO']
0 []
1 ['R3UO8GRRT862EQ']
0 []
1 ['RBCABL8RGM9GR']
0 []
1 ['R3CPPEA292B27X']
0 []
1 ['R31FEBE2UMRQYA']
0 []
0 []
1 ['R3S06NI78ZYFXM']
1 ['RWASLN3ORPDA7']
0 []
1 ['R11SXF0TU9LXHD']
0 []
0 []
1 ['R2CNRCJOZ06J01']
1 ['RRF0FKF7G9M3M']
0 []
1 ['R3MV150C1UK5NZ']
0 []
1 ['R17FGAFEQ74XZY']
0 []
1 ['R2A08CI9IIR98G']
0 []
0 []
1 ['R32DSBW68CM09G']
1 ['R2I4G5C28OUQP4']
0 []
1 ['R3N1C32TEG8ZZ4']
0 []
1 ['R1F2BVYIHRVBXK']
0 []
1 ['RLYQ8B240YTFK']
0 []
1 ['R2BTVCG6CN0Z1B']
0 []
0 []
1 ['RLXTLD5VTX858']
1 ['R141F4EIZ54IJ6']
0 []
1 ['R3REM0UBSV9OUW']
0 []
0 []
1 ['R14TWYGC1CXPFO']
1 ['R3VC1L0ENF80PP']
0 []
0 []
1 ['R3RE969VU7IU4C']
1 ['R1QPNXF052JSS6']
0 []
1 ['R3NQ1AENZL3D6I']
0 []
1 ['R2DQ264MLYPTGU']
0 []
1 ['R3EH46MWBPHRWA']
0 []
0 []
1 ['R31WST0V9KMLC8']
0 []
1 ['R3SAZCLNUMTLOR']
1 ['R32DR2FBLG4VK0']
0 []
1 ['RSIBE5QASCAV']
0 []
1 ['R55PPPNW069L5']
0 []
0 []
1 ['R3IUSYCQ77RW22']
1 ['RAN0MPNVJ8ODC']
0 []
0 []
1 ['R34DAFEI3N9W90']
1 ['R30RTB2FLBD1HL']
0 []
1 ['R17RT9YAHGPLML']


0 []
1 ['R2MV8R7D6XRR1A']
1 ['R316R9YZ3PQM31']
0 []
1 ['R4VJKRIVNPD4Z']
0 []
1 ['R3SBVUXI42Z3FL']
0 []
0 []
1 ['R2EHJ7H6SLMPXJ']
1 ['R3DJQQY0MOKU35']
0 []
1 ['R3A6JL6RSK7XV8']
0 []
0 []
1 ['R360NUU31VAGZN']
1 ['R1L0FTUMJNAG49']
0 []
1 ['R1THQ2ZDW4ZNI7']
0 []
1 ['R2WS2DZIWCQG0P']
0 []
1 ['R9VSD0ET8FERB']
0 []
1 ['R110H8261M3AVK']
0 []
1 ['R1LKEBHGJLH4R2']
0 []
1 ['R2SQJ8C0U96TLG']
0 []
1 ['R1O634T1WUVP8P']
0 []
1 ['RT32VMT62M7PR']
0 []
0 []
1 ['R3TA5B3RUSYL0Y']
0 []
1 ['R3R0K8EYGBMPQ5']
0 []
1 ['R2EUJGBJ8S6AB6']
0 []
1 ['RM35UV4RRYK6T']
1 ['R2010W1RAPZ3WL']
0 []
0 []
1 ['R5VX5RY4CWFLK']
0 []
1 ['R3GQ82K9GQ7JFH']
1 ['R23LXKV01PSA39']
0 []
0 []
1 ['RUJA4K0YSC0AI']
1 ['R3PSS4JDXJEV0O']
0 []
1 ['R3QM4ZF2IX8ORI']
0 []
1 ['R3G5WE99H4TWIG']
0 []
0 []
1 ['RL4PQ7R05YXTN']
1 ['R1FJ2DLYAA7LHX']
0 []
0 []
1 ['RPOJDQLCOPJ0A']
0 []
1 ['R3IX0VSTC82G6I']
0 []
1 ['R8GFZ1X7YJ6PZ']
1 ['R3RU28ZIP97UDS']
0 []
1 ['R3IP40C4JUOAZK']
0 []
0 []
1 ['R13CK83NL3NYA7']
1 ['RWRD4W8EG76J1']
0 []
1 ['R677V1YQ2R3R0']
0 

...

1 ['RBZZL0160E1BJ']
0 []
1 ['R1VPD0ECUXCL1G']
0 []
1 ['R11W0ZT4F3NTMJ']
0 []
1 ['R1JL63HHUPAUCY']
0 []
1 ['R3T5K2LDSOWY5W']
0 []
0 []
1 ['R4MPKWT20PN38']
0 []
1 ['R38PEL1D9CKKM5']
0 []
1 ['RRZPMYKAZ68IJ']
1 ['R1YMECJ02TGJTM']
0 []
1 ['RF7XIT673NFYO']
0 []
1 ['R2OYMFPFTVMUXA']
0 []
0 []
1 ['R18G56NE4R2LTO']
1 ['R1SR64J2FAE7O7']
0 []
0 []
1 ['R1L84GTLUWFLRV']
1 ['R3BTQGOWM8TRKC']
0 []
1 ['R3AB3ONF54BLRA']
0 []
0 []
1 ['R3PP5R7COW9HSW']
0 []
1 ['RX5HGC46IO6M9']
1 ['R2Y8MBKGY5PJ5L']
0 []
1 ['R2SUTVMG982CAI']
0 []
1 ['R1SQYECHDII07B']
0 []
1 ['R2O6HCGLOUYHUZ']
0 []
0 []
1 ['R2IRC6GFJWK19K']
0 []
1 ['R267KJEAQLAYH7']
1 ['R213XKFYEQDTZS']
0 []
0 []
1 ['R35IGRKXGUJGDO']
1 ['R2Q247QS8QWG2G']
0 []
1 ['RQ9U7ACGBS5GX']
0 []
1 ['R3VZKLZ288C2SO']
0 []
0 []
1 ['R2HIQG1YIKIH0G']
0 []
1 ['R15H91QC8J0F9Z']
0 []
1 ['R1ZYHETTZ5LV99']
1 ['R3H3CBR83MJWKH']
0 []
0 []
1 ['R37OPAN4JT7PVT']
0 []
1 ['RLYRNRJV4S3I0']
0 []
1 ['R2YE0DXUB41IXX']
0 []
1 ['R38NBV6MRQ3G18']
0 []
1 ['RO8M21S94NS1I']
1 ['R2W192WLVBAKHH']

1 ['RUHCFRC53LU4Z']
0 []
1 ['R1V6LPPA9D55JB']
0 []
0 []
1 ['R2WEY6E771IBFL']
0 []
1 ['R115VRQGRHTTL3']
1 ['R1ACJOQ4IGYREF']
0 []
1 ['R1XFZL2JOBY3IN']
0 []
1 ['R17GCR8XH5B715']
0 []
1 ['R2AUNMFYYE7GNC']
0 []
1 ['R18B2X3BDI3TP0']
0 []
1 ['ROIZOR0MTN4ZE']
0 []
0 []
1 ['RS7VYWQR5IYKN']
0 []
1 ['RMWBJ81IFUHYT']
0 []
1 ['RSK5KJV9829M0']
1 ['R2KTSPZPZGLWAZ']
0 []
1 ['R2K2GWOZULA160']
0 []
0 []
1 ['R2QYKYBHMM9559']
0 []
1 ['RZV0LQ7HEFJOB']
1 ['R3TLIBFK3HB2OY']
0 []
1 ['R3F0HFAVXFL8C5']
0 []
0 []
1 ['R1Q3X8SIZRNVRD']
1 ['R3R9RZ713PME5P']
0 []
1 ['R5IWGJG763M18']
0 []
0 []
1 ['R3NF4D5C3K6A7O']
1 ['R87DY2OKIBAT7']
0 []
1 ['R3U2D1XBTNPKK9']
0 []
0 []
1 ['RMJKVZNXKQ4ER']
0 []
1 ['R13KFFG75T5IW0']
1 ['R2V4L3XC8WBI4T']
0 []
1 ['R2E87TKUMOP61L']
0 []
1 ['R1CZ5IT4IP0MFR']
0 []
1 ['R1ETORA49OTLVQ']
0 []
1 ['R2TAVTJKLWBA39']
0 []
1 ['R2QKFE14WK022F']
0 []
1 ['R2OVTSQBHM1L2Z']
0 []
1 ['R184JHH4AGCEAE']
0 []
1 ['R3FMU5XV3BKV43']
0 []
0 []
1 ['R1L57UZ38LS3DP']
1 ['RKBSK752K66K4']
0 []
0 []
1 ['R1KUMUMXVIY8D

0 []
1 ['R2SZF2P62S353O']
1 ['R2OX3CU4LH0FA3']
0 []
1 ['R9P12M6K2ANJG']
1 ['R2OB9VYXXU8NG6']
1 ['R2CQHIBR2W1UTY']
0 []
1 ['R2I137QIVGLE1O']
0 []
1 ['R160OMGJM4VEX6']
0 []
1 ['RUVV4X3J8DB27']
0 []
1 ['R2DOYL30D9VBH2']
0 []
1 ['R1UD1F76P2Y59O']
0 []
1 ['R161WJSZD96032']
0 []
1 ['R382QWNP2FOKZ2']
0 []
1 ['R185IB3A18S2Z0']
0 []
0 []
1 ['R2Z8CFH3LE1IV']
1 ['RNEPTTGRRUYBB']
0 []
0 []
1 ['R3I7M6VR70REAA']
0 []
1 ['R17Q8LA6SOWL9H']
1 ['R3003U32ICB1QD']
0 []
0 []
1 ['R23UKT7009BI1M']
0 []
1 ['R110OODOR932AB']
1 ['R2HZGMP1BQBQ35']
0 []
1 ['RJNGGCFAQOGSB']
0 []
1 ['R18I55RUQ8PGMG']
0 []
0 []
1 ['RYYU2VEDMSSN0']
1 ['R1MPRX0XJID0AK']
0 []
1 ['R1LLYL76UTYWU5']
0 []
0 []
1 ['RUP7Q0O7QBAYL']
1 ['R8TW6E0N6MC39']
0 []
0 []
1 ['RUEAFEER372JW']
1 ['R1924CFAP7W6ED']
0 []
1 ['R1RDXVIIEAT7TD']
0 []
1 ['R1TQH3TL3VFMXX']
0 []
1 ['RWWFE78N0VSX1']
0 []
1 ['RVC89ZKMWXG12']
0 []
1 ['R3D5PQS7E0ILAZ']
0 []
1 ['R2NUE5GBUWMD9H']
0 []
1 ['RFZ7ZZ4EQBZ0T']
0 []
1 ['R306F1QQB66XFP']
0 []
1 ['RCRZLFJG94MOD']
0 []
0 []
1 ['

1 ['RJEN6UO1F271O']
0 []
1 ['R17RKCGKZX21ID']
0 []
1 ['R2VDHA16RGD77Q']
0 []
1 ['R2T5BBS0BSB0F4']
0 []
0 []
1 ['RJO0SDB4CQGK0']
1 ['RDEASUKXIW2DX']
0 []
1 ['R174JEPQYDBI09']
0 []
1 ['R5JSOZ2QM54XL']
0 []
0 []
2 ['RQ8I1VY47G37R', 'R2CDN3ZQTQCXBP']
1 ['R1YTFT6DUB7PIF']
0 []
1 ['R2Y067SIASDO2J']
0 []
0 []
1 ['R29CGY9814XZC8']
1 ['R13L1I4Q9MH1GW']
1 ['RS6HR505MQ5OF']
0 []
1 ['R3P1LJ9JOSZIF9']
1 ['RPX7M8RPMF5HR']
0 []
1 ['R2VUVOI2K1CR4T']
0 []
1 ['R39XQAYS79A13K']
0 []
1 ['R1HQTZ7RJB9UW8']
0 []
0 []
1 ['RB3E8EGY0V0SZ']
1 ['R23ONONLN8OTL1']
0 []
1 ['RXDJXEJVUGMZZ']
0 []
0 []
1 ['R3S0SD1R6HXPRR']
1 ['RIUA2RYK23CVC']
0 []
1 ['R1MO26712BD26I']
0 []
1 ['R2FYD0N7LVB1LZ']
0 []
1 ['RSU0QAOFSIQKB']
0 []
1 ['RO7P4TB02SOP3']
0 []
0 []
1 ['R2C0GT5BKKSB65']
1 ['R9RGCN4RKI0EP']
0 []
1 ['R2N67M1WMEPVD5']
0 []
0 []
1 ['R18RMABCTQ2TH1']
0 []
1 ['R339XOONMWLROW']
1 ['R35NQM9FBXVFAA']
0 []
0 []
1 ['R3FOXOR29S5VW7']
1 ['R2PSWNTV7XQTWV']
0 []
1 ['RRTCE3HFH20ML']
0 []
1 ['RD8869546SSK1']
0 []
1 ['RWCO5JT1E8Z94']

KeyError: nan

dict_keys(['2015-09-31'])

## 5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

In [5]:
dataset = pd.read_csv("reviews2.csv",sep="\t")

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#5. Reviews(>3) from same IP on the same day with all the reviews are either positive or negative.

ip_group = dataset.groupby("IP Address")
# grouping the dataset by ip addresses

ip_list = dataset["IP Address"].unique().tolist()
# stores the list of unique ip addresses

size = len(ip_list)
# total no of unique ip addresses

for i in range(size):
    # iterate through all the ip addresses
    
    reviews = ip_group.get_group( ip_list[i] )
    # dataframe of each ip
    
    dates_list = reviews["review_date"].unique().tolist()
    # list of dates of reviews by each ip addresses
    
    reviews_by_date = reviews.groupby("review_date");
    # grouping the dataframe by date
    
    for j in range(len(dates_list)):
        # iterate through all the dates
        
        reviews_by_date_for_pos = []
        reviews_by_date_for_neg = []

        reviews_for_each_day = reviews_by_date.get_group(dates_list[j])
        #dataframe of reviews for a day by each ip addresses
        
        indices = reviews_for_each_day.index.tolist()
        # list of indices of the dataframe reviews_for_each_day
        
        for k in range(len(reviews_for_each_day)):
            #iterate through all the reviews on a day by each ip addresses
            
            text = reviews_for_each_day["review_body"][ indices[k] ]
            # reviews on a day for an ip addresses
            
            if(getSentiment(text) == 0):
                
                #if sentiment is negative, append review_id to list of negative reviews
                reviews_by_date_for_neg.append(reviews_for_each_day["review_id"][ indices[k] ])
            else:
                
                #if sentiment is positive, append review_id to list of positive reviews
                reviews_by_date_for_pos.append(reviews_for_each_day["review_id"][ indices[k] ])
                
        # CONDITION FOR CONSIDERING THE FAKE REVIEW        
        print(len(reviews_by_date_for_pos) , reviews_by_date_for_pos)
        print(len(reviews_by_date_for_neg) , reviews_by_date_for_neg)
        # count, review id of reviews with same sentiment
        
        

7 ['R8MEA6IGAHO0B', 'R31LOQ8JGLPRLK', 'R2Y0MM9YE6OP3P', 'RRB9C05HDOD4O', 'R31LOQ8JGLPRLK', 'R31LOQ8JGLPRLK', 'R31LOQ8JGLPRLK']
0 []
5 ['R26I2RI1GFV8QG', 'RY8DDL22YG4R5', 'R2AT2426ZHFUHH', 'R8MEA6IGAHO0B', 'R26I2RI1GFV8QG']
0 []
3 ['R2Y0MM9YE6OP3P', 'R2Y0MM9YE6OP3P', 'R2Y0MM9YE6OP3P']
0 []
1 ['RRB9C05HDOD4O']
2 ['RRB9C05HDOD4O', 'RRB9C05HDOD4O']
2 ['R8MEA6IGAHO0B', 'R8MEA6IGAHO0B']
0 []
1 ['RY8DDL22YG4R5']
0 []
1 ['R2AT2426ZHFUHH']
0 []
1 ['R3RRXU2R23NMQ9']
0 []
1 ['R250PR8VJUZ62F']
0 []
1 ['RBEMQ29WJBHYG']
0 []
1 ['R19VVIUT4BZCMT']
0 []
0 []
1 ['R1DT8JJUQHUKUL']
1 ['RLIL6S9OGM3YA']
0 []
1 ['R34EZZ68VYPHO0']
0 []
0 []
1 ['R3TZNSOXS13SIG']
1 ['R12UBZT87UX3AP']
0 []
0 []
1 ['R5KCRE927NYMU']
1 ['RT12RQFU0V2AX']
0 []
0 []
1 ['R2K0YQPV5W2SJ1']
1 ['R104MY31OKNGXH']
0 []
1 ['R3EQYRGIIJQBHT']
0 []
1 ['R32MW6D6CXKCFW']
0 []
1 ['RFO6FJSAA391Z']
0 []
1 ['R1IG9KI4EWNE4W']
0 []
1 ['R3IF1H9L7WSGQD']
0 []
1 ['R38T4K7X8UBVO7']
0 []
1 ['R19C76D4LWB6CM']
0 []
0 []
1 ['R260G1C72KG6KC']
1 ['R2SLX39DAJSG0V'

1 ['RHOR6RVMS370J']
0 []
0 []
1 ['R3MGZ75QFPTB0S']
0 []
1 ['R30DV9DJ4YDSEH']
0 []
1 ['R1VMJENF098LBW']
1 ['R7L22LEG2BUVA']
0 []
0 []
1 ['R3QS7HA9QRDIZ9']
1 ['RRLR0UYQ5HEBQ']
0 []
0 []
1 ['R1JP6MNSIT0U2D']
1 ['R1UUWXSTSYBDRA']
0 []
1 ['R3TOTF7UMXG7PK']
0 []
0 []
1 ['R3MAHRZX4TVR7C']
1 ['R1AEL8XMKRT511']
0 []
0 []
1 ['R7WBC0JAA9UI']
1 ['R3BTDRVS0QUH91']
0 []
1 ['R2KXEAPUMU14IE']
0 []
1 ['R34Z5GSD6QVJC0']
0 []
0 []
1 ['R396KCIONQ5GMH']
0 []
1 ['R1RZE8IT05J6D8']
0 []
1 ['R3VJXA3GLDX8E9']
0 []
1 ['R2IJUP8JD5W160']
1 ['RYV0E9CUARKQO']
0 []
0 []
1 ['R30XZYR7CNQLJQ']
1 ['RX2TJPSKVFAR2']
0 []
1 ['RVQ6QWBRBKCNU']
0 []
1 ['R9BSZTOXA2VP6']
0 []
1 ['RL98J8W0EYXF8']
0 []
1 ['R3KD1E9HEX42AL']
0 []
1 ['R1GA8ZRKXIEABY']
0 []
1 ['RUF3XGO1TYY5M']
0 []
1 ['R7NFBSNDY7NI9']
0 []
0 []
1 ['RH6DCVWOCMMRZ']
1 ['R3G3I8VVGG5UQR']
0 []
1 ['R2OCDOIHJVF0EJ']
0 []
1 ['R49B90CZF8G9K']
0 []
1 ['R1PD6K9907WGKH']
0 []
0 []
1 ['R1IJODCM3KMBPZ']
1 ['R3Q0LBAFCM37G4']
0 []
0 []
1 ['R18FK3K0X0ENPK']
1 ['R199SSULGKZ4TV']
0 []


1 ['R115ACF5NH8VCT']
0 []
1 ['R2CNZSHYDNZLF']
0 []
1 ['R2B1J0DV4Q90C']
0 []
1 ['RTV7V99DW7LAR']
0 []
1 ['R3QUX6UDEI7VEJ']
0 []
0 []
1 ['R2OHETRELQHI3L']
1 ['R1MU62FFMJ4URR']
0 []
1 ['R20EQ2PZ10UC5Q']
0 []
1 ['R1MUN0TZJC9MSA']
0 []
1 ['R2JR1SFS2V4RXR']
0 []
1 ['R2R51UWKLR517C']
0 []
1 ['R1NKGBEN0I4D01']
0 []
1 ['R1SDQJBNNFGHFW']
0 []
0 []
1 ['R1XJTQUONRQDDL']
1 ['R2V9RVGPI8U2FA']
0 []
1 ['R23WA4EC8WVMD9']
0 []
1 ['R2H5CL0LU2KH3Q']
0 []
1 ['R26OXBIFZKCCI8']
0 []
1 ['R5M17UM979QXF']
0 []
1 ['RNN38RFJVIE5G']
0 []
1 ['R1DTSELYJEDK2M']
0 []
0 []
1 ['R1XOP8ERFM4B5E']
1 ['R3MWLDA9M77DJ5']
0 []
1 ['R1RPRSVD6QYAX1']
0 []
1 ['R1S6OGES5HTB55']
0 []
1 ['RTIOX61PKJVN1']
0 []
0 []
1 ['R877I6M0MKQ97']
1 ['R30WJHSATH8ESG']
0 []
1 ['RE0MBYEZJR5TA']
0 []
1 ['R1ZQMYGHOEUHL7']
0 []
1 ['R38O0DB0BLUWJT']
0 []
1 ['R2BB2MQN08YUHK']
0 []
1 ['R3V7GO2GYDJA9M']
0 []
0 []
1 ['R2E16WLZKP78GO']
0 []
1 ['R273W61K6FDZIH']
0 []
1 ['R16DHVUT6YROK8']
0 []
1 ['R18WYZ5QGOHZQF']
1 ['R2K4XXTLWCLHDA']
0 []
0 []
1 ['R2ANYVYEXRX

0 []
1 ['RUFSYWR4JEM9X']
0 []
1 ['R1UEL9NYZZGW6T']
1 ['R3897EN7VFWT9H']
0 []
1 ['R30UCDE2BLWOOO']
0 []
1 ['R1C7SGB3B0HHTU']
0 []
1 ['R2QL75BY562O1N']
0 []
1 ['R291PG86PP3Z1G']
0 []
1 ['R3KX5WKLAVXPTO']
0 []
1 ['R3UO8GRRT862EQ']
0 []
1 ['RBCABL8RGM9GR']
0 []
1 ['R31FEBE2UMRQYA']
0 []
0 []
1 ['R3S06NI78ZYFXM']
1 ['RWASLN3ORPDA7']
0 []
1 ['R11SXF0TU9LXHD']
0 []
0 []
1 ['R2CNRCJOZ06J01']
1 ['RRF0FKF7G9M3M']
0 []
1 ['R3MV150C1UK5NZ']
0 []
1 ['R17FGAFEQ74XZY']
0 []
1 ['R2A08CI9IIR98G']
0 []
1 ['R1BMBUGPTM393I']
0 []
0 []
1 ['R32DSBW68CM09G']
1 ['R2I4G5C28OUQP4']
0 []
1 ['R3N1C32TEG8ZZ4']
0 []
1 ['R1F2BVYIHRVBXK']
0 []
1 ['RLYQ8B240YTFK']
0 []
1 ['R2BTVCG6CN0Z1B']
0 []
0 []
1 ['RLXTLD5VTX858']
1 ['R141F4EIZ54IJ6']
0 []
1 ['R3REM0UBSV9OUW']
0 []
0 []
1 ['R14TWYGC1CXPFO']
1 ['R3VC1L0ENF80PP']
0 []
0 []
1 ['R3RE969VU7IU4C']
1 ['R1QPNXF052JSS6']
0 []
1 ['R3NQ1AENZL3D6I']
0 []
1 ['R26BEG5J79Q7WU']
0 []
1 ['R2DQ264MLYPTGU']
0 []
1 ['R3EH46MWBPHRWA']
0 []
0 []
1 ['R31WST0V9KMLC8']
0 []
1 ['R3SAZCLNU

0 []
1 ['R256GKL8SJTXEZ']
1 ['R3V1LOA01MJO96']
0 []
1 ['R16PNF5FYHUDMK']
0 []
1 ['R3O923V2PRBBM4']
0 []
0 []
1 ['R11YQH9NSDFVQ3']
1 ['RG1A8ZSUHRKCJ']
0 []
0 []
1 ['R2MV8R7D6XRR1A']
1 ['R316R9YZ3PQM31']
0 []
1 ['R4VJKRIVNPD4Z']
0 []
1 ['R3SBVUXI42Z3FL']
0 []
0 []
1 ['R2EHJ7H6SLMPXJ']
1 ['R3DJQQY0MOKU35']
0 []
1 ['R3A6JL6RSK7XV8']
0 []
0 []
1 ['R360NUU31VAGZN']
1 ['R1L0FTUMJNAG49']
0 []
1 ['R1THQ2ZDW4ZNI7']
0 []
1 ['R2WS2DZIWCQG0P']
0 []
1 ['R9VSD0ET8FERB']
0 []
1 ['R110H8261M3AVK']
0 []
1 ['R1LKEBHGJLH4R2']
0 []
1 ['R2SQJ8C0U96TLG']
0 []
1 ['R1O634T1WUVP8P']
0 []
1 ['RT32VMT62M7PR']
0 []
0 []
1 ['R3TA5B3RUSYL0Y']
0 []
1 ['R3R0K8EYGBMPQ5']
0 []
1 ['R2EUJGBJ8S6AB6']
0 []
1 ['RM35UV4RRYK6T']
1 ['R2010W1RAPZ3WL']
0 []
0 []
1 ['R5VX5RY4CWFLK']
0 []
1 ['R3GQ82K9GQ7JFH']
1 ['R23LXKV01PSA39']
0 []
0 []
1 ['RUJA4K0YSC0AI']
1 ['R3PSS4JDXJEV0O']
0 []
1 ['R3QM4ZF2IX8ORI']
0 []
1 ['R3G5WE99H4TWIG']
0 []
0 []
1 ['RL4PQ7R05YXTN']
1 ['R1FJ2DLYAA7LHX']
0 []
0 []
1 ['RPOJDQLCOPJ0A']
0 []
1 ['R3IX0VSTC82G

...

1 ['R1P7VERHYSFMSA']
0 []
0 []
1 ['R37DVVKNQSX1FN']
1 ['R2HDAP3MZ2D390']
0 []
1 ['R1WRF899VUW2TG']
0 []
1 ['R3KH9T6WOPHINX']
0 []
1 ['R2VOVBL4OT00F0']
0 []
0 []
1 ['R2U6LSJTXJH4AE']
0 []
1 ['R2Y6WHXOUCRCSR']
0 []
1 ['RP5JCQUMY589R']
1 ['R1KH270TI9ZXAG']
0 []
0 []
1 ['R2ELU9ZZRQSO68']
1 ['R171TY5E5GRSZ4']
0 []
1 ['R7J66KD3LHP50']
0 []
0 []
1 ['R2AWPZSJU4HGKO']
1 ['R2IFGYQP051YXX']
0 []
1 ['R29SB7WAP3SIRF']
0 []
1 ['R2Y7KX6CHKM4RI']
0 []
1 ['R3VJ13SZOCHQJM']
0 []
1 ['RBZZL0160E1BJ']
0 []
1 ['R1VPD0ECUXCL1G']
0 []
1 ['R11W0ZT4F3NTMJ']
0 []
1 ['R1JL63HHUPAUCY']
0 []
1 ['R3T5K2LDSOWY5W']
0 []
0 []
1 ['R4MPKWT20PN38']
0 []
1 ['R38PEL1D9CKKM5']
0 []
1 ['RRZPMYKAZ68IJ']
1 ['R1YMECJ02TGJTM']
0 []
1 ['RF7XIT673NFYO']
0 []
1 ['R2OYMFPFTVMUXA']
0 []
0 []
1 ['R18G56NE4R2LTO']
1 ['R1SR64J2FAE7O7']
0 []
0 []
1 ['R1L84GTLUWFLRV']
1 ['R3BTQGOWM8TRKC']
0 []
1 ['R3AB3ONF54BLRA']
0 []
0 []
1 ['R3PP5R7COW9HSW']
0 []
1 ['RX5HGC46IO6M9']
1 ['R2Y8MBKGY5PJ5L']
0 []
1 ['R2SUTVMG982CAI']
0 []
1 ['R1SQYECHDII07B'

0 []
1 ['RSK5KJV9829M0']
1 ['R2KTSPZPZGLWAZ']
0 []
1 ['R2K2GWOZULA160']
0 []
0 []
1 ['R2QYKYBHMM9559']
1 ['R3TLIBFK3HB2OY']
0 []
1 ['R3F0HFAVXFL8C5']
0 []
0 []
1 ['R1Q3X8SIZRNVRD']
1 ['R3R9RZ713PME5P']
0 []
1 ['R5IWGJG763M18']
0 []
0 []
1 ['R3NF4D5C3K6A7O']
1 ['R87DY2OKIBAT7']
0 []
1 ['R3U2D1XBTNPKK9']
0 []
0 []
1 ['RMJKVZNXKQ4ER']
0 []
1 ['R13KFFG75T5IW0']
1 ['R2V4L3XC8WBI4T']
0 []
1 ['R2E87TKUMOP61L']
0 []
1 ['R1CZ5IT4IP0MFR']
0 []
1 ['R1ETORA49OTLVQ']
0 []
1 ['R2TAVTJKLWBA39']
0 []
1 ['R2QKFE14WK022F']
0 []
1 ['R2OVTSQBHM1L2Z']
0 []
1 ['R184JHH4AGCEAE']
0 []
1 ['R3FMU5XV3BKV43']
0 []
0 []
1 ['R1L57UZ38LS3DP']
1 ['RKBSK752K66K4']
0 []
0 []
1 ['R1KUMUMXVIY8DL']
1 ['R1DGT2X918BCD9']
0 []
1 ['RRU0S3DIVPOP4']
0 []
1 ['R2IRNSRMOCNFK9']
0 []
1 ['R1OCHAQ1A7BJZF']
0 []
1 ['R2IYEQL4JE6C2W']
0 []
0 []
1 ['R2JRYXDLW8D8XJ']
1 ['R3242S9WUJ9ZXM']
0 []
1 ['R2IOKHA9BBRYAF']
0 []
0 []
1 ['RWG8P89XTMWNR']
1 ['R2R4V5MS8EDG5Z']
0 []
1 ['R1K63HL46LFFQ5']
0 []
0 []
1 ['R9I5G8HW66IB3']
1 ['R3QFPGS166LMQT']

1 ['R3003U32ICB1QD']
0 []
0 []
1 ['R23UKT7009BI1M']
0 []
1 ['R110OODOR932AB']
1 ['R2HZGMP1BQBQ35']
0 []
1 ['RJNGGCFAQOGSB']
0 []
1 ['R18I55RUQ8PGMG']
0 []
0 []
1 ['RYYU2VEDMSSN0']
1 ['R1MPRX0XJID0AK']
0 []
1 ['R1LLYL76UTYWU5']
0 []
0 []
1 ['RUP7Q0O7QBAYL']
1 ['R8TW6E0N6MC39']
0 []
0 []
1 ['RUEAFEER372JW']
1 ['R1924CFAP7W6ED']
0 []
1 ['R1RDXVIIEAT7TD']
0 []
1 ['R1TQH3TL3VFMXX']
0 []
1 ['RWWFE78N0VSX1']
0 []
1 ['RVC89ZKMWXG12']
0 []
1 ['R3D5PQS7E0ILAZ']
0 []
1 ['R2NUE5GBUWMD9H']
0 []
1 ['RFZ7ZZ4EQBZ0T']
0 []
1 ['R306F1QQB66XFP']
0 []
1 ['RCRZLFJG94MOD']
0 []
0 []
1 ['RMN7D345T46EW']
1 ['R3OTJRG1QNUXJF']
0 []
1 ['R1AWLJFRX9HPKL']
0 []
0 []
1 ['R33Q5GU236BJAQ']
1 ['R3LWN4E8IIFRBB']
0 []
1 ['R3ILRRL9A1OHSX']
0 []
1 ['R3BN3LBNMNKBN1']
0 []
1 ['R2APB3XR6EMJB0']
0 []
0 []
1 ['R4OGNFHSJBDPG']
0 []
1 ['R3FU447IOVA4EA']
0 []
1 ['RF92SF1CWISI1']
1 ['R2DMV2AMZQBF1C']
0 []
0 []
1 ['RA3YFM101J29F']
1 ['R1CUDZ97B36IK']
0 []
1 ['RB24G70R8VQ7X']
0 []
0 []
1 ['RJPQNT2HQNM7P']
1 ['RODZFB96E4Q6Z']
0 []
1 [

0 []
1 ['RJO0SDB4CQGK0']
1 ['RDEASUKXIW2DX']
0 []
1 ['R174JEPQYDBI09']
0 []
1 ['R5JSOZ2QM54XL']
0 []
0 []
1 ['RQ8I1VY47G37R']
1 ['R1YTFT6DUB7PIF']
0 []
1 ['R2Y067SIASDO2J']
0 []
0 []
1 ['R29CGY9814XZC8']
0 []
1 ['RS6HR505MQ5OF']
0 []
1 ['R3P1LJ9JOSZIF9']
1 ['RPX7M8RPMF5HR']
0 []
0 []
1 ['RB3E8EGY0V0SZ']
1 ['R23ONONLN8OTL1']
0 []
1 ['RXDJXEJVUGMZZ']
0 []
0 []
1 ['R3S0SD1R6HXPRR']
1 ['RIUA2RYK23CVC']
0 []
1 ['R1MO26712BD26I']
0 []
1 ['R2FYD0N7LVB1LZ']
0 []
1 ['RO7P4TB02SOP3']
0 []
0 []
1 ['R2C0GT5BKKSB65']
1 ['R9RGCN4RKI0EP']
0 []
1 ['R2N67M1WMEPVD5']
0 []
0 []
1 ['R18RMABCTQ2TH1']
0 []
1 ['R339XOONMWLROW']
1 ['R35NQM9FBXVFAA']
0 []
0 []
1 ['R3FOXOR29S5VW7']
1 ['R2PSWNTV7XQTWV']
0 []
1 ['RRTCE3HFH20ML']
0 []
0 []
1 ['R2CDN3ZQTQCXBP']
1 ['RD8869546SSK1']
0 []
1 ['RWCO5JT1E8Z94']
0 []
0 []
1 ['RYRMWX8WE79D4']
1 ['R17WJ7T5U4M7XG']
0 []
0 []
1 ['RWIOB8ZYNDQZB']
1 ['R1J94QBU5K5SWF']
0 []
1 ['RGOCYS5F8D6EY']
0 []
0 []
1 ['R1H0OEJKS9BUSD']
1 ['R9V0KASK3QJ30']
0 []
1 ['R91S8MDCJSJ1']
0 []
1 ['R1

KeyError: nan

In [ ]:
## 6th pending

## 7. Removing reviews with no. of verbs > no. of nouns

In [4]:
dataset = pd.read_csv("reviews2.csv",sep='\t')
dataset.head(2)

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address,timestamp
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,207.156.210.106,1.441005e+09
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,193.93.167.87,1.441038e+09


In [6]:
#7. Removing reviews with no. of verbs > no. of nouns

fake_review = []
# store the list of review_id of fake reviews

for i in range(len(dataset)):
    #iterate the whole dataset
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    #storing the words from the reviews into the list
    
    tagged_words = nltk.pos_tag(words)
    # returns list of tuples of words along with their parts of speech
    
    nouns_count = 0
    verbs_count = 0
    
    for j in range(len(tagged_words)):
        #iterate through all the words

        if(tagged_words[j][1].startswith("NN")):
            nouns_count+=1
            #counts the no. of nouns in the review

        if(tagged_words[j][1].startswith("VB")):
            verbs_count+=1
            #counts the no. of verbs in the review

    if(verbs_count>nouns_count):
        #comparing the no. of verbs and nouns
        fake_review.append(dataset["review_id"][i])
        #storing the 
        
fake_review

['R8MEA6IGAHO0B',
 'R31LOQ8JGLPRLK',
 'RRB9C05HDOD4O',
 'R26I2RI1GFV8QG',
 'R2AT2426ZHFUHH',
 'RLIL6S9OGM3YA',
 'RT12RQFU0V2AX',
 'R1IG9KI4EWNE4W',
 'R260G1C72KG6KC',
 'R31SFE3UEUA0EY',
 'R174DGBVW0UWAJ',
 'R2074W1A28UUU6',
 'R3B3E2E0123J3F',
 'R3VUB98IS1VKQC',
 'R18TG2USOSVOC3',
 'R2DWIADFBQ3IYG',
 'R2VMBHZ9XTC0GQ',
 'R27GI77VF2P6WF',
 'R35NN211X0RGH9',
 'R18MK5UOS41XJN',
 'R2K2WK38XR5FKZ',
 'R3FBUNWANUJRTU',
 'RGAH8RF1K8QUI',
 'R3KOQ03SYO1FM2',
 'R2TMCO418IRILU',
 'R2H1V2O1SVWULZ']

## 8. Removing reviews with includes more first person pronouns.

In [ ]:
# 8. Removing reviews with includes more first person pronouns.

fake_reviews = []
#store the list of review_id of reviews

for i in range(len(dataset)):
    #iterate through all the reviews
    
    dataset["review_body"][i] = dataset["review_body"][i].lower()
    # converting each characters to its lower cases
    
    words = nltk.word_tokenize(dataset["review_body"][i])
    # storing the list of words for each reviews
    
    sentence = nltk.sent_tokenize(dataset["review_body"][i])
    # storing the list of sentences for each reviews
    
    count=0
    if(len(sentence)>4):
        # Checking only those reviews which have atleast 5 sentences.
        
        for j in range(len(words)):
            #iterating through all the reviews
            
            if(words[j]=="i" or words[j]=="we"):
                #counting personal pronouns
                count+=1
                
        if(count > len(sentence)/2):
            #reviews with number of personal pronouns used greater than half the no. of sentences.
            fake_reviews.append(dataset["review_id"][i])
fake_reviews

/home/anubhav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
